In [15]:
import sys
sys.path.append("..")  # Add the parent directory to the path

import os 
import torch 
import numpy as np
from torch import Tensor
from typing import Dict, List, Optional, Tuple, Union
from gsplat.rendering import rasterization
import yaml
from gsplat.strategy.default import DefaultStrategy
from munch import munchify 

In [22]:
def load_ckpt(ckpt_path : str, device="cuda"):
    assert os.path.isfile(ckpt_path), f"checkpoint not found in {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=True)
    return ckpt

def read_splats(ckpt):
    """ function to read the splats from a pre-trained checkpoint
        input  : pytorch checkpooint
        output : dictionary of splats """
    splats = {key : ckpt['splats'][key] for key in ckpt['splats'].keys()}
    return splats 

# Define a constructor function for the DefaultStrategy
def default_strategy_constructor(loader, node):
    # Here, you can extract any necessary data from the node
    # and use it to initialize the DefaultStrategy object.
    # For simplicity, we'll assume no additional data is needed.
    return DefaultStrategy()

# Register the constructor with PyYAML
yaml.add_constructor('tag:yaml.org,2002:python/object:gsplat.strategy.default.DefaultStrategy', default_strategy_constructor)

def read_config(config_path):
    assert os.path.isfile(config_path), f"configuration did not found in {config_path}"
    with open(config_path, 'r') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)  # Use yaml.FullLoader to handle Python-specific tags
    return config

ckpt = load_ckpt("/home/sergio/onboarding_stage/gaussian_splatting/results/hope/obj_000001/ckpts/ckpt_14999_rank0.pt")
splats = read_splats(ckpt)
points_obj = np.asarray(splats['means'].cpu())
config = read_config("/home/sergio/onboarding_stage/gaussian_splatting/results/hope/obj_000001/cfg.yml")
cfg = munchify(config)

### compute cameras along icosphere 

In [23]:
def load_point_cloud(file_path):
    """
    Dummy point-cloud loading function.
    Replace with your own code to load real data (e.g. .ply or .xyz).
    Returns:
        points: (N, 3) numpy array
    """
    # For illustration, we generate random points in a box
    # Remove or replace this with your real data loading
    points = np.random.rand(1000, 3) * 2.0 - 1.0  # random points in [-1, 1]^3
    return points

def compute_centroid_and_radius(points):
    """
    Compute the centroid (mean of all points) and a radius
    proportional to the object size. 
    
    Options for the radius:
    1) Half of the maximum bounding-box extent (simple bounding sphere).
    2) An actual minimal bounding sphere (more involved).
    Here we do the bounding box approach for simplicity.
    """
    centroid = np.mean(points, axis=0)
    
    # bounding box extents
    min_xyz = np.min(points, axis=0)
    max_xyz = np.max(points, axis=0)
    bbox_size = max_xyz - min_xyz
    radius = 0.5 * np.linalg.norm(bbox_size)  # half-diagonal
    
    return centroid, radius

def icosahedron_vertices():
    """
    Return the base icosahedron vertices (12) and faces (20).
    The returned vertices are on the unit sphere.
    """
    phi = (1.0 + np.sqrt(5.0)) / 2.0  # golden ratio
    # Twelve vertices of an icosahedron
    verts = np.array([
        [-1,  phi,  0],
        [ 1,  phi,  0],
        [-1, -phi,  0],
        [ 1, -phi,  0],
        [ 0, -1,  phi],
        [ 0,  1,  phi],
        [ 0, -1, -phi],
        [ 0,  1, -phi],
        [ phi,  0, -1],
        [ phi,  0,  1],
        [-phi,  0, -1],
        [-phi,  0,  1]
    ], dtype=np.float64)
    
    # Normalize to unit sphere
    verts /= np.linalg.norm(verts, axis=1)[:, None]
    
    # Faces of the icosahedron (20 triangular faces)
    faces = np.array([
        [0, 11, 5], [0, 5, 1], [0, 1, 7], [0, 7, 10], [0, 10, 11],
        [1, 5, 9], [5, 11, 4], [11, 10, 2], [10, 7, 6], [7, 1, 8],
        [3, 9, 4], [3, 4, 2], [3, 2, 6], [3, 6, 8], [3, 8, 9],
        [4, 9, 5], [2, 4, 11], [6, 2, 10], [8, 6, 7], [9, 8, 1]
    ], dtype=np.int32)
    
    return verts, faces

def subdivide_icosahedron_once(verts, faces):
    """
    Subdivide each triangle in the icosahedron once.
    This yields 42 unique vertices on the unit sphere.
    """
    edge_map = {}
    new_faces = []
    next_vertex_index = len(verts)
    # We’ll store new vertices in a list to append them as we generate them
    new_verts = verts.tolist()
    
    def get_midpoint_index(i1, i2):
        """
        For the edge (i1, i2), compute the midpoint on the unit sphere.
        Use a dictionary to avoid duplicating edges.
        """
        # Ensure smaller index first for consistent edge key
        if i2 < i1:
            i1, i2 = i2, i1
        edge_key = (i1, i2)
        if edge_key in edge_map:
            return edge_map[edge_key]
        
        # Compute midpoint and normalize
        v1 = np.array(new_verts[i1])
        v2 = np.array(new_verts[i2])
        midpoint = 0.5 * (v1 + v2)
        midpoint /= np.linalg.norm(midpoint)
        
        # Add to new_verts list
        new_verts.append(midpoint.tolist())
        edge_map[edge_key] = len(new_verts) - 1
        return edge_map[edge_key]
    
    # Subdivide each face
    for tri in faces:
        iA, iB, iC = tri[0], tri[1], tri[2]
        iAB = get_midpoint_index(iA, iB)
        iBC = get_midpoint_index(iB, iC)
        iCA = get_midpoint_index(iC, iA)
        
        # Create new faces
        new_faces.append([iA, iAB, iCA])
        new_faces.append([iB, iBC, iAB])
        new_faces.append([iC, iCA, iBC])
        new_faces.append([iAB, iBC, iCA])
    
    new_verts = np.array(new_verts, dtype=np.float64)
    new_faces = np.array(new_faces, dtype=np.int32)
    
    return new_verts, new_faces

def generate_icosphere_level_1():
    """
    Generates an icosphere at subdivision level 1.
    This should have 42 vertices on the unit sphere.
    """
    base_verts, base_faces = icosahedron_vertices()
    verts, faces = subdivide_icosahedron_once(base_verts, base_faces)
    # Optionally, you can remove duplicates if needed, but with the edge_map approach,
    # we should already have unique vertices.
    return verts, faces

def look_at(camera_pos, target, up=np.array([0, 1, 0], dtype=float)):
    """
    Compute a standard right-handed look-at camera extrinsic matrix:
    
    - camera_pos: 3D position of the camera.
    - target: 3D position to look at.
    - up: approximate "world up" vector.
    
    Returns:
        A 4x4 extrinsic matrix (world->camera).
        The camera looks down the -Z axis in its local coordinate system.
    """
    forward = camera_pos - target
    forward /= np.linalg.norm(forward)  # The -Z axis in camera coords
    
    # Recompute orthonormal basis
    right = np.cross(up, forward)
    right /= (np.linalg.norm(right) + 3e-10)
    
    up_new = np.cross(forward, right)
    up_new /= (np.linalg.norm(up_new) + 3e-10)
    
    # Rotation part
    R = np.eye(4, dtype=float)
    # Camera’s X axis = 'right'
    R[0, 0:3] = right
    # Camera’s Y axis = 'up'
    R[1, 0:3] = up_new
    # Camera’s Z axis = 'forward' (which is -Z from camera’s perspective)
    R[2, 0:3] = forward
    
    # Translation part
    T = np.eye(4, dtype=float)
    T[0:3, 3] = -camera_pos
    
    # The extrinsic matrix is R * T
    extrinsic = R @ T
    
    return extrinsic

    
# 2) Compute centroid and radius
centroid, radius = compute_centroid_and_radius(points_obj)
    
# 3) Generate the icosphere of 42 vertices
icosphere_verts, _ = generate_icosphere_level_1()
    
# 4) Scale and translate icosphere to the centroid with the chosen radius
#    Each vertex becomes a camera position
camera_positions = icosphere_verts * radius + centroid
    
# 5) For each camera position, compute the extrinsic matrix looking at the centroid
camera_poses = []
for cam_pos in camera_positions:
    pose = look_at(cam_pos, centroid)
    camera_poses.append(pose)
    
# Print (or store) the results
print(f"Number of camera poses: {len(camera_poses)}")


Number of camera poses: 42


In [24]:
world_size = 1
def rasterize_splats(cfg,
    splats : dict,
    camtoworlds: Tensor,
    Ks: Tensor,
    width: int,
    height: int,
    masks: Optional[Tensor] = None,
    **kwargs,
) -> Tuple[Tensor, Tensor, Dict]:
    means = splats["means"]  # [N, 3]
        # quats = F.normalize(self.splats["quats"], dim=-1)  # [N, 4]
        # rasterization does normalization internally
    quats = splats["quats"]  # [N, 4]
    scales = torch.exp(splats["scales"])  # [N, 3]
    opacities = torch.sigmoid(splats["opacities"])  # [N,]

    # image_ids = kwargs.pop("image_ids", None)
    colors = torch.cat([splats["sh0"], splats["shN"]], 1)  # [N, K, 3]

    rasterize_mode = "antialiased" if cfg.antialiased else "classic"
    render_colors, render_alphas, info = rasterization(
        means=means,
        quats=quats,
        scales=scales,
        opacities=opacities,
        colors=colors,
        viewmats=torch.linalg.inv(camtoworlds),  # [C, 4, 4]
        Ks=Ks,  # [C, 3, 3]
        width=width,
        height=height,
        packed=cfg.packed,
        absgrad=(
            cfg.strategy.absgrad
            if isinstance(cfg.strategy, DefaultStrategy)
            else False
        ),
        sparse_grad=cfg.sparse_grad,
        rasterize_mode=rasterize_mode,
        distributed=world_size > 1,
        camera_model=cfg.camera_model,
        **kwargs,
    )
    if masks is not None:
        render_colors[~masks] = 0
    return render_colors, render_alphas, info

In [21]:
camera_pose = camera_poses[0]
renders, alphas, info = rasterize_splats(
    camtoworlds=camt,
    Ks=Ks,
    width=width,
    height=height,
    sh_degree=sh_degree_to_use,
    near_plane=cfg.near_plane,
                far_plane=cfg.far_plane,
                image_ids=image_ids,
                render_mode="RGB+ED" if cfg.depth_loss else "RGB",
                masks=masks,
            )